In [12]:
import requests
from time import sleep
import re
import pymongo
import threading
import os.path

In [13]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [14]:
def save_pic(filename,url):
    if(os.path.exists('filename')):
        return
    try:
        response = requests.session().get(url, stream=True)
        with open(filename,"w+b") as f:
            f.write(response.content)
    except requests.exceptions.ContentDecodingError as e:
        print(type(e))
    except Exception as e1:
        print(e1)
        save_pic(filename,url)
        

In [15]:
class QzoneCrawler:
    def __init__(self,QQnumber="",pwd="",filename="posts.csv"):
        self.qq=QQnumber
        self.pwd=pwd        
        self.csv_file_writer=open(filename,"w+",1)
        client = pymongo.MongoClient('localhost', 27017)
        db = client['posts-database']
        self.posts=db['posts']
        self.setup()
        client.close()
    def save_to_db(self,post):
        try:
            self.posts.insert_one(post)
        except Exception as e:
            print(e)
        
    def setup(self):
        options = Options()
        caps=webdriver.DesiredCapabilities.FIREFOX
        caps["pageLoadStrategy"]="eager"
        self.driver = webdriver.Firefox(options=options,capabilities=caps)
        self.driver.get("http://www.qzone.com")
        self.retrieve_cookie()
        sleep(5)
        self.driver.refresh()
        sleep(5)
        e=WebDriverWait(self.driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR , "a[title=说说]")))
        e.click()
        iframe=WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR , ".app_canvas_frame")))
        self.driver.switch_to.frame(iframe)        
        
        final_page = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR , "#pager_last_0>span")))
        final_page= int(final_page.text)
  
        for i in range(final_page):
            self.retrieve_page(i+1)
      
    def retrieve_cookie(self):
        #不保障有验证码
        print("LOG IN")
        try:
            WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR , "iframe")))
            self.driver.switch_to.frame("login_frame")
            WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR , "a#switcher_plogin"))).click()
            WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR , "input[type=text]"))).send_keys(self.qq)
            WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR , "input[type=password]"))).send_keys(self.pwd)
            WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR , "#login_button"))).click()
            self.driver.switch_to.default_content()
        except Exception as e:
            print(e)
            
    def retrieve_page(self,page_num):
        try:
            input_id="input.textinput"
#             input_id="#pager_go_"+str(page_num-1)
            print(input_id)
            e=WebDriverWait(self.driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR , input_id)))
            e.location_once_scrolled_into_view
            e.send_keys(str(page_num))
            e.send_keys(Keys.RETURN)

            for i in range(4):
                WebDriverWait(self.driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR , input_id))).location_once_scrolled_into_view
            sleep(2)
            li=self.driver.find_elements_by_class_name("feed")
            last_content=None
            for content in li:
                content.location_once_scrolled_into_view
                more_link=content.find_element_by_class_name("has_more_con")
                if (more_link.text)=="展开查看全文":
                    more_link.location_once_scrolled_into_view
                    self.driver.execute_script("arguments[0].click();", more_link)
                    sleep(3)
                e=(content.find_element_by_class_name("content"))
                data={}
                data["_id"]=content.get_attribute("data-tid")
                print(data["_id"])
                data["content"]=e.text
                data["images"]=list()


                imgs=(content.find_elements_by_css_selector(".md img"))
                for img in imgs:
                    while re.search("blank.gif$",img.get_attribute("src"))!=None:
                        sleep(2)
                        WebDriverWait(self.driver, 10).until(
                            EC.presence_of_element_located((By.CSS_SELECTOR , "#QM_Mood_Poster_Container"))).location_once_scrolled_into_view
                        sleep(2)
                        img.location_once_scrolled_into_view    
                        sleep(2)
                        if last_content!=None:
                            last_content.location_once_scrolled_into_view
                        sleep(2)
                        img.location_once_scrolled_into_view                        
                last_content=content

                for i in range(len(imgs)):
                    filename=data["_id"]+'_'+str(i)+'.jpg'
                    t = threading.Thread(target=save_pic, args=('./images/'+filename,img.get_attribute("src")))
                    t.start()
                    data["images"].append(filename)
                self.save_to_db(data)
        except Exception as e:
            print("RETRY")
            self.retrieve_page(page_num)
                

In [16]:
qc=QzoneCrawler("492992365","password")

LOG IN
input.textinput
6d77621db001235caaec0000
6d77621d4ffaf05b93cf0300
6d77621d9bcde15b41e20600
6d77621de6cadf5b643d0400
6d77621dbe02dc5b7b760c00
6d77621de50abb5b44150f00
6d77621d2ec5b95b74440300
6d77621d5c47b95b05310200
6d77621db2e7b35bae3a0300
6d77621de873ac5bf0e40000
6d77621dc08aab5b7c3d0600
6d77621dd3a2a35bc6b70900
6d77621d7522a35be5fd0900
6d77621db79c985b22f00e00
6d77621dcc24955b82980c00
6d77621dc81d905b03320400
6d77621db1428f5b2bad0a00
6d77621d4eaf885b2faf0300
6d77621ddc6b865bc5750900
6d77621da0cf835b437c0a00
input.textinput
6d77621d5a0b7a5bf75e0800
6d77621da0766f5bb09d0e00
6d77621d7ede6b5b57b10400
6d77621df656655b70090d00
6d77621df62a605be0660500
6d77621d5f425c5ba57c0400
6d77621de91b5c5bc64f0800
6d77621d9cd24c5be1b50200
6d77621d7b724b5b6bc80b00
6d77621d8230445bc4810b00
6d77621d61ea2e5b5e380800
6d77621d50a1285b23110b00
6d77621d0ade255be17e0600
6d77621d1e4a245b50c60300
6d77621df456235b00e60100
6d77621da3331e5bfffc0e00
6d77621ddab01c5b1a320200
6d77621dce110e5b4a7c0200
6d77621d61c

6d77621d56b239598c100f00
6d77621d1785395903210b00
6d77621ddfb2385999460b00
6d77621def8e385984c50e00
6d77621d69ed345944e40400
6d77621dbe8d3359dc070200
6d77621d8c84335946fe0400
6d77621d32153259cf270e00
6d77621de30d325952470e00
6d77621d00822c5930280d00
6d77621d72a22b59ebee0500
6d77621de23b2b591c2c0900
6d77621d912f2759202b0a00
6d77621d49992459837e0600
6d77621d72b22359388c0b00
6d77621d3ba5235903190600
6d77621de0eb1f5934870500
6d77621d2ecb1f59a5c30000
input.textinput
6d77621d81a91f59fa6a0200
6d77621d2ea61f59d5d50800
6d77621dff6c1a59c0060800
6d77621ddd091a595a470300
6d77621ddc7b1659f0eb0100
6d77621db358155996600a00
6d77621d7d471559a0e20900
6d77621d052b1559d1d10000
6d77621da91d1459c1f10100
6d77621d0a070c5937de0700
6d77621d89a60a59bebd0100
6d77621de4510959a1c30e00
6d77621d2eb00659c6530600
6d77621d7599065917a20600
6d77621d75a50559de840e00
6d77621d69f5025915b50800
6d77621d9c8302590c310e00
6d77621d50e3fe5832e10500
6d77621d92e1fe58dc2d0a00
6d77621d6794fd58de640200
input.textinput
6d77621d05f2f8588e

6d77621da76b2f5815480e00
input.textinput
6d77621dd79c2e58931c0100
6d77621d27792e5814180f00
6d77621d09792e588fc10600
6d77621d384d2e58e4f90700
6d77621dbe702d58b8680b00
6d77621d7b122c58c7e20900
6d77621d4bb82b58e18d0d00
6d77621dc1cd2a5801a60300
6d77621d0b792958675a0300
6d77621dadc92658ea190c00
6d77621d4f852658c61f0500
<class 'requests.exceptions.ContentDecodingError'>
6d77621dff3725588ed90400
6d77621ddd33245878750c00
6d77621d0edc2258eba60000
6d77621dee8c215800b80500
6d77621de1272058cfea0300
6d77621d6ca21e585e090800
6d77621dd9561e58064f0b00
6d77621d00101c584f610900
6d77621d68da185853760b00
input.textinput
RETRY
input.textinput
RETRY
input.textinput
6d77621d5ed4155800130700
6d77621daaaa155842e60900
6d77621d1aa41558c0550100
6d77621d4717155861b20d00
6d77621de8801458169e0700
6d77621d6b351058f9900e00
6d77621df8d40f5816bc0700
6d77621d25550d5828870500
6d77621d3c930c58a2ce0c00
6d77621dbe110b58b2a20600
6d77621d028c09583ac00c00
6d77621dcbd2075875bb0200
6d77621d750b075889960400
6d77621d518b0458d4d7050

6d77621d5b15fe5602bc0b00
input.textinput
6d77621d7211fe56190d0d00
6d77621dc204fe56c7560900
6d77621d6b89fd56d67d0300
6d77621d43e0fa5668800400
6d77621d854dfa5690dc0c00
6d77621d4d72f756efb30b00
6d77621d8807f756457b0700
6d77621d01e6f5567b990300
6d77621dd9ebf45669c30600
6d77621d27c9f456a1380600
6d77621d5498f4567ccd0000
6d77621d749bf35620fc0100
6d77621d27faf256471f0600
6d77621d884bf2561faf0900
6d77621dc839f2568de80500
6d77621dea37f256461d0900
6d77621da6e4f0562bfb0600
6d77621d45b6f056ccb90800
6d77621de1b9ef5667b40900
6d77621dd8d0ed5646d50a00
input.textinput
6d77621da3f4ec5688ff0400
6d77621d6d68ec56e41a0000
6d77621d9e9eeb56a98f0400
6d77621db897eb5612b20600
6d77621dc875ea5662db0d00
6d77621d3b2cea568c120500
6d77621d390ee95666b00400
6d77621d94eee856ebd50900
6d77621d1613e656bcba0600
6d77621dd211e656a8540700
6d77621da5ffe456e95b0800
6d77621d33f2e35607160100
6d77621dcfd4e356dcfc0300
6d77621d5313e356978c0200
6d77621dd074e256ddb30e00
6d77621d966be256b40d0000
6d77621db992e0568deb0c00
6d77621d3fb6df568b

6d77621dfe756755e2af0500
6d77621df6286455fa6b0c00
6d77621d49c36255a2d40e00
6d77621dc4bf6255dcb80400
6d77621d20a562553a490c00
input.textinput
6d77621ddb6c62558a3d0700
6d77621de93c625537360e00
6d77621da1806155d4d40200
6d77621ddb5661551ba50300
6d77621d532e6155afb20600
6d77621d06256155fea60300
6d77621d4ee96055f4830900
6d77621d873b5e5511140b00
6d77621d9a5f5d55d15b0100
6d77621da091595539970d00
6d77621d47bc575559890b00
6d77621d0fa85755509e0b00
6d77621d20535455e1ef0200
6d77621d01055455a9480600
6d77621dacb3535564dd0200
6d77621d8ee152551a330500
6d77621db5c5525534780500
6d77621d611a515573190700
6d77621df96350557ba20900
6d77621d2d924e553b6b0000
input.textinput
6d77621da2934d555d330c00
6d77621d808d4d5526350600
6d77621d9c624d5507850300
6d77621d51504d55acc30900
6d77621d4bac495594770c00
6d77621d62eb47551a640200
6d77621d4ffb465534de0500
6d77621d04f6465583fe0400
6d77621df1f446551bbd0400
6d77621dc65f445525700500
6d77621db5bd3d5557520a00
6d77621d5f683d55e5e30100
6d77621d62143d55278c0400
6d77621d00683c5581

...

6d77621de55a935015460700
input.textinput
6d77621d834f93508ef00800
6d77621d0b379350f6f60a00
6d77621d101793500df80200
6d77621d90f292501f600d00
6d77621d781e9250dc2b0900
6d77621d78169250f25a0500
6d77621ded12925057260b00
6d77621d1fe59150ec2a0b00
6d77621dbde09150006d0200
6d77621dc79291504bf70c00
6d77621d66b790503b990700
6d77621d1fb390501fad0800
6d77621dacae9050e8340400
6d77621d23ac905034f00100
6d77621db4ab90505b630b00
6d77621d199f9050384a0100
6d77621d4f8f9050dfd10100
6d77621de1798f50e5430b00
6d77621d0a4b8f5088700300
6d77621ddc2f8e50aa960300
input.textinput
6d77621dac2b8e5002760c00
6d77621ddc038e5049630c00
6d77621dd1fd8d50b0b00a00
6d77621dd8e98d5081170900
6d77621de8cb8d500c210600
6d77621dd0d68c50b0520500
6d77621d71bf8c5093340300
6d77621dd3bd8c50878b0a00
6d77621db4aa8c50f23e0500
6d77621dee358b50f39e0500
6d77621d99358b505e190d00
6d77621dce318b501c5f0500
6d77621dc1de895061ba0900
6d77621d17de8850cfc70400
6d77621d4cdb88503b0e0300
6d77621df9b688505c110000
6d77621d6a8a8750b6f60100
6d77621de556875024

6d77621db209225078850a00
6d77621dc4f62150b6290b00
input.textinput
6d77621dc4dd21502f0f0b00
6d77621dc1d12150908e0100
6d77621d19d0215075880100
6d77621daecf2150090b0c00
6d77621de77f2150f9f60500
6d77621d0f9720502e920c00
6d77621d7a8d205061d00a00
6d77621d1b832050f4910100
6d77621d4a722050fc100f00
6d77621d943b2050bbf80c00
6d77621d0230205064cc0100
6d77621df4771f50d0660100
6d77621d6a771f50b65c0600
6d77621d28751f509dbe0d00
6d77621d056c1f50f0960000
6d77621dba511f50b0fa0700
6d77621d461a1e509a510700
6d77621dd5f41d50644c0d00
6d77621d25f31d5075a00a00
6d77621d7ba41d50df1f0300
input.textinput
6d77621d78921d50e6f00600
6d77621d6b821d50a2330b00
6d77621d93b51c5058ff0300
6d77621dfbb21c503ab60600
6d77621db8a71c50ebee0e00
6d77621d7b401c50721d0300
6d77621d8c771b50e96b0300
6d77621d6fe81a50720e0500
6d77621dd2211a50c0a90100
6d77621d31211a5073df0700
6d77621d6b051a50ae930d00
6d77621dd6d7185040540d00
6d77621db0c418502e1c0300
6d77621d73c3185023220100
6d77621decc218503b440b00
6d77621dcc3b1750cb020900
6d77621dd03a17507e

6d77621d035c834f32720800
6d77621d5e81824f82f90900
6d77621d1481824f3f250100
6d77621d546f824fd5f60d00
input.textinput
6d77621daf6d824fcf510300
6d77621d81f2814f44290400
6d77621df9fd804f1b9b0800
6d77621dfe1a7f4f19b30500
6d77621da8607e4f23ca0e00
6d77621d354c7d4fde730600
6d77621d14297d4fd6b10900
6d77621def1c7d4fd48c0c00
6d77621dab1c7d4fc95f0900
6d77621df00a7d4f5fdb0d00
6d77621df7dc7c4f2ea70d00
6d77621d655c7a4f90820800
6d77621d9de2794f05c40d00
6d77621d4232794ff25e0000
6d77621d67cf784fe56b0e00
6d77621d91c9784f5e9c0200
6d77621d44bf784f262b0200
6d77621dfa94784f49dd0900
6d77621da0d6774fb26a0b00
6d77621d25d6774fd6cd0a00
input.textinput
6d77621d0782764f532a0700
6d77621db57f764fd9480600
6d77621d0f61764f71cb0900
6d77621d2045754f79500700
6d77621d4a32754f763a0300
6d77621dc62d754fc83e0200
6d77621ddb27754fccb40e00
6d77621d6b21754f83b40e00
6d77621d6ef6744fd3d40d00
6d77621d93de744f41c70100
6d77621df902744f34550e00
6d77621d0fd8734f5a490200
6d77621dc6c8734f512c0600
6d77621dfc87724f81bf0900
6d77621d857f724f5a

6d77621d01c3fe4e96fe0e00
6d77621d0417fd4ea49a0900
6d77621dbc0bfa4e2dba0600
6d77621db70bfa4e91600b00
6d77621d77f5f84eb8120800
input.textinput
6d77621d7fdff84e39a80400
6d77621d3b27f84e168c0700
6d77621d5576f54e03e00d00
6d77621de80ff54ebc470300
6d77621dad0cf44e72550300
6d77621d81edf34e24950300
6d77621d73edf34e19210c00
6d77621da8e0f34ee8cd0300
6d77621da0e0f34e827f0000
6d77621dcd57f14ea5010400
6d77621dc757f14e165f0300
6d77621dad7fed4e6d790100
6d77621da517ec4e52380100
6d77621d6914ec4e64f30200
6d77621d9ea1e94e154c0000
6d77621d7857e94eb7cb0d00
6d77621d4e3fe84e7e8c0800
6d77621d393fe84e118c0400
6d77621db90ce84e53c30400
6d77621dea06e44e03490700
input.textinput
6d77621d32f5e24ee2bb0800
6d77621db38de14efbc30100
6d77621db65de14e9a790000
6d77621d9cf2df4e22e10100
6d77621da707df4ef85d0400
6d77621d0d9ede4e9c150f00
6d77621d029ede4e1ef40000
6d77621d2abfdd4e3fbe0e00
6d77621d4248dc4eef990500
6d77621d4627dc4ef7c10c00
6d77621de4d4da4eaf280f00
6d77621d548cda4e9fcd0400
6d77621d1e87da4e55720e00
6d77621d84c7d94e60

6d77621d89477c4d1a930700
6d77621d52437b4d9e270700
6d77621d7adf794d5bf80900
6d77621dabdd784ddad70b00
6d77621d14db784dbf230d00
6d77621dc9da784dad0c0900
6d77621d8ebd784d01580000
input.textinput
6d77621d84b7784d2c290300
6d77621d5988784df8ff0b00
6d77621d6151774da0170000
6d77621daf45774d36db0400
6d77621d6d44764d08740000
6d77621dbc34764d130e0c00
6d77621d2414764d9cda0e00
6d77621dfeef754dc70a0c00
6d77621d8dbc744d45730900
6d77621d8c2a734df5820900
6d77621dc414734d8b8d0500
6d77621d67f7724d162a0500
6d77621db75b724d555f0600
6d77621d37d6714dea950b00
6d77621d5b9d704d5c460d00
6d77621da4676f4dbc360500
6d77621dd2176f4d10ee0b00
6d77621df2c46c4dc0310300
6d77621db4c46c4da4a10000
6d77621de97f6b4d9d180500
input.textinput
6d77621d92dd694d7e490200
6d77621db8a5674d62470d00
6d77621d1c7a674dfb940e00
6d77621d294d664d78b90e00
6d77621dcf05654d4fcf0e00
6d77621de3a5634d7c070d00
6d77621dcea5634dae470c00
6d77621dc6a5634dad110900
6d77621dc1a5634de5ff0800
6d77621d1da5634df3960800
6d77621d85a4634dee260200
6d77621d6f6a624d59

True